# Building an AI Email Assistant with Mistral AI and Composio

This notebook demonstrates how to build an AI-powered email assistant using Mistral AI and Composio in less than 50 lines of code! By using Mistral’s large language models (LLMs) and Composio’s Gmail tool, we can create a system capable of performing common Gmail actions such as sending, fetching, drafting emails etc., all through natural language commands.

## Setup and Dependencies

### Mistral AI API

Sign up at [Mistral AI](https://mistral.ai/) and generate an API key from the console. This key will be used to access Mistral's LLM models.

In [ ]:
MISTRAL_API_KEY="your-mistral-api-key"

### Composio Setup

Follow these steps to set up a Composio integration for Gmail -  

1. Sign up for [Composio](https://app.composio.dev/), and browse to the Tools catalogue page from the sidebar.
2. Search for the Gmail tool and select it.
3. Select 'Setup Gmail Integration', click 'Save' and complete the authentication flow.
4. Done! You have set-up an integration with Gmail.

### Python Libraries

Install the necessary libraries to set up the AI agent - 

In [ ]:
%pip install composio_langchain composio_core langchain-mistralai 

## Building the Email Assistant

### Initialize Composio's tools

The first thing we do is initialize Composio's tools.

In [ ]:
from composio_langchain import Action, ComposioToolSet

toolset = ComposioToolSet()
tools = toolset.get_actions(
    actions=[
        Action.GMAIL_SEND_EMAIL, 
        Action.GMAIL_FETCH_EMAILS, 
        Action.GMAIL_CREATE_EMAIL_DRAFT, 
        Action.GMAIL_CREATE_LABEL
    ],
)

The various actions represent functions that the LLM can call. In order for Mistral models to understand the functions, we need to outline the function specifications with a JSON schema. Specifically, we need to describe the type, function name, function description, function parameters, and the required parameter for the function. Composio does this and provides an optimized JSON schema, making it very easy to integrate external tools with LLMs. For example, for the `GMAIL_SEND_EMAIL` action, this is the schema provided to the model - 

```
{
	"properties":{
		"recipient_email":{
			"description":"Email address of the recipient",
			"examples":["john@doe.com"],
			"title":"Recipient Email",
			"type":"string"
			},
			"subject":{
				"description":"Subject of the email",
				"examples":["meeting"],
				"title":"Subject",
				"type":"string"
			},
			"body":{
				"description":"Body content of the email",
				"examples":["Hey, nice talking to you"],
				"title":"Body",
				"type":"string"
			},
			"user_id":{
				"default":"me",
				"description":"The user's email address or 'me' for the authenticated user.",
				"title":"User Id",
				"type":"string"
			}
		},
		"required":[
			"recipient_email",
			"subject",
			"body"
		],
		"title":"CreateEmailDraftRequest",
		"type":"object"
}

### Setting up the model



In [ ]:
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatMistralAI(model="mistral-large-latest")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI email assistant that can write, fetch, and manage emails and labels. Follow the user’s instructions carefully and perform the requested actions. If fetching emails, print the Subject and Mail content in a readable format.",
        ),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

We then provide the task we want the agent to do - 

In [ ]:
task = input("Please enter the task you want to perform: ")

We then set up an agent to execute the task - 

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": task})
print(response)

Please enter the task you want to perform: Send a mail to example@gmail.com saying hi                           
The email has been sent successfully to example@gmail.com. 

## Conclusion

In this guide, we built an AI email assistant that can perform various tasks by combining Mistral’s LLMs with Composio’s prebuilt actions. By following the steps outlined in this guide, you can extend this framework to other applications, creating a versatile tools that leverage the power of LLMs and function calling.